# 📊 Sales Analytics Storytelling Dashboard
## Turning Transactional Data into Strategic Growth Insights

**Project Context**: This analysis uses synthetic sales data to demonstrate how tracking core sales KPIs can reveal growth opportunities, highlight customer loyalty trends, and optimize product mix profitability.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from datetime import datetime

# Set plotting theme
import plotly.io as pio
pio.templates.default = "plotly_white"
pio.renderers.default = "notebook_connected"

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 1. Executive Summary: The High-Level Pulse
Before diving into the details, let's look at the 'vital signs' of our sales performance: Revenue, AOV, and Churn.

In [2]:
# Load pre-calculated KPIs
kpi_path = '../outputs/sales_kpis.csv'
kpi_df = pd.read_csv(kpi_path)

# Load raw sales for trend analysis
sales = pd.read_csv('../../shared/data/sales_transactions.csv')
sales['date'] = pd.to_datetime(sales['date'])

def get_kpi(metric_name):
    return kpi_df[kpi_df['metric'] == metric_name]['value'].values[0]

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = get_kpi('Revenue Growth Rate (Last Month)'),
    title = {"text": "Revenue Growth %"},
    delta = {'reference': 5, 'relative': False, 'position': "bottom"},
    domain = {'x': [0, 0.3], 'y': [0, 1]}),
    row=1, col=1
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = get_kpi('Average Order Value (AOV)'),
    number = {'prefix': "$"},
    title = {"text": "Avg Order Value"},
    domain = {'x': [0.35, 0.65], 'y': [0, 1]}),
    row=1, col=2
)

fig.add_trace(go.Indicator(
    mode = "number",
    value = get_kpi('Customer Churn Rate (90-day)'),
    number = {'suffix': "%"},
    title = {"text": "Churn Rate"},
    domain = {'x': [0.7, 1], 'y': [0, 1]}),
    row=1, col=3
)

fig.update_layout(height=300, title_text="Executive Sales Snapshot")
fig.show()

## 2. Revenue Trends: Momentum Analysis
Is our growth steady, or are we relying on holiday spikes? Understanding the monthly velocity helps in resource planning.

In [3]:
monthly_rev = sales.set_index('date').resample('ME')['revenue'].sum().reset_index()

fig = px.line(monthly_rev, x='date', y='revenue', 
              title='Monthly Revenue Progression',
              labels={'revenue': 'Total Revenue ($)', 'date': 'Month'},
              template='plotly_white')

fig.add_bar(x=monthly_rev['date'], y=monthly_rev['revenue'], name='Revenue', opacity=0.3)
fig.update_traces(line=dict(color='#2E86C1'), selector=dict(type='scatter'))
fig.show()

## 3. Product Mix: Where is the Profit?
Not all revenue is created equal. Some categories drive volume, while others drive margin.

In [4]:
cat_perf = sales.groupby('product_category').agg({
    'revenue': 'sum',
    'profit': 'sum',
    'transaction_id': 'count'
}).reset_index()

cat_perf['margin_pct'] = (cat_perf['profit'] / cat_perf['revenue']) * 100
cat_perf = cat_perf.rename(columns={'transaction_id': 'orders'})

fig = px.scatter(cat_perf, x='revenue', y='margin_pct', 
                 size='orders', color='product_category',
                 hover_name='product_category',
                 title='Profitability vs. Volume by Category',
                 labels={'revenue': 'Total Revenue ($)', 'margin_pct': 'Profit Margin (%)'},
                 text='product_category')

fig.update_traces(textposition='top center')
fig.add_hline(y=cat_perf['margin_pct'].mean(), line_dash="dot", annotation_text="Avg Margin")
fig.show()

## 4. Customer Segmentation: The 80/20 Rule
Which customer segments are our 'Whales' (High Value) and which are just 'Browsers'?

In [5]:
seg_data = sales.groupby('customer_segment').agg({
    'revenue': 'sum',
    'customer_id': 'nunique'
}).reset_index()

seg_data['rev_per_cust'] = seg_data['revenue'] / seg_data['customer_id']

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'bar'}]])

fig.add_trace(go.Pie(labels=seg_data['customer_segment'], values=seg_data['revenue'], name="Revenue Share"), 1, 1)
fig.add_trace(go.Bar(x=seg_data['customer_segment'], y=seg_data['rev_per_cust'], name="Revenue per Customer"), 1, 2)

fig.update_layout(title_text="Customer Segment Value Analysis")
fig.show()

## 5. Strategic Insights & Recommendations
This section summarizes the performance against targets.

In [6]:
insights_text = """
1. **Growth Momentum**: Monthly revenue is {0}% {1} compared to last month. {2}
2. **Margin Opportunities**: The **{3}** category has the highest profit margin ({4}%), while **{5}** drives the most volume.
3. **Customer Retention**: A churn rate of **{6}%** is {7}.
4. **AOV Strategy**: With an AOV of **${8}**, we are {9} our target.
""".format(
    round(abs(get_kpi('Revenue Growth Rate (Last Month)')), 1),
    "up" if get_kpi('Revenue Growth Rate (Last Month)') > 0 else "down",
    "Keep the momentum!" if get_kpi('Revenue Growth Rate (Last Month)') > 5 else "Investigation needed.",
    cat_perf.loc[cat_perf['margin_pct'].idxmax(), 'product_category'],
    round(cat_perf['margin_pct'].max(), 1),
    cat_perf.loc[cat_perf['revenue'].idxmax(), 'product_category'],
    round(get_kpi('Customer Churn Rate (90-day)'), 1),
    "concerning" if get_kpi('Customer Churn Rate (90-day)') > 20 else "healthy",
    round(get_kpi('Average Order Value (AOV)'), 2),
    "meeting" if get_kpi('Average Order Value (AOV)') > 500 else "below"
)

from IPython.display import Markdown
display(Markdown(insights_text))


1. **Growth Momentum**: Monthly revenue is 19.1% up compared to last month. Keep the momentum!
2. **Margin Opportunities**: The **Home & Garden** category has the highest profit margin (45.4%), while **Electronics** drives the most volume.
3. **Customer Retention**: A churn rate of **29.8%** is concerning.
4. **AOV Strategy**: With an AOV of **$824.22**, we are meeting our target.
